### Carga diaria de Analisis de Sentimientos

*  Cargar los datos
 * Cargar TRM Historica y calcular deltas
*  Calcular correlaciones spearman y pearson
 * Correlaciones con diferentes Lags hasta 15 días
* Validar otras variables

In [37]:
import pandas as pd
import tweepy
from datetime import *
import datetime as dt
import json, re
import feedparser
from time import mktime

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
import mysql.connector
connection = mysql.connector.connect(host='localhost', port=3307,
                                         database='tesis',
                                         user='root',
                                         password='Cidifo03%%')

In [12]:
# Analisis de Sentimientos
##librerias ingles
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


#librerías Español
import joblib
#import sklearn.external.joblib as extjoblib
from  classifier import SentimentClassifier
#from classifier import *
clf = SentimentClassifier()

# importaciones Tokenizacion

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
pst = PorterStemmer()
import string
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
stopWEnglish = set(stopwords.words('english'))
stopWSP = set(stopwords.words('spanish'))

from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
pd.set_option("display.max_colwidth", 100)

In [36]:
#fechaHoy=datetime.datetime.now().date()
fechaHoy=dt.datetime.now().date()

#  Funciones

In [15]:
def limpiar_terminos(texto):    
    texto=" ".join(filter(lambda x:x[0]!='#', texto.split()))
    texto=" ".join(filter(lambda x:x[0]!='@', texto.split()))
    texto=" ".join(filter(lambda x:x[0:4]!='http', texto.split()))
    return texto

In [16]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """    
    texto=limpiar_terminos(texto)
    
    # Eliminamos los caracteres especiales
    texto = re.sub(r'\W', ' ', str(texto))
    texto = re.sub(r'\d', ' ', str(texto))
    # Eliminamos comas y /
    texto = re.sub(r',', ' ', str(texto))
    texto = re.sub(r'/', ' ', str(texto))
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    # remover \n
    #texto = re.sub(r'\n', '', texto)
    texto = texto.replace('\n', '')
    texto = texto.replace('º', '')    
      
    # Convertimos textos a minusculas
    texto = texto.lower()
    return texto

In [17]:
def analisisSentimientosIngles(text):
    resultado = sid.polarity_scores(text)
    return resultado["compound"]

## enEspañol
def analisisSentimientosEspanol(text):
    resultado= clf.predict(text)
    resultado= resultado*2-1
    #resultado= resultado
    return resultado


def seleccionAnalisis(fila):
    #print(fila)
    if fila["lang"] == "en" :
        return analisisSentimientosIngles(str(fila["full_text"]))
    elif fila["lang"] == "es" :
        return analisisSentimientosEspanol(str(fila["full_text"]))

#tokenizacion
def tokenizcionIngles(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"''"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    textMin = word_tokenize(textPunt.lower())
    textSW = [x for x in textMin if x not in stopWEnglish]
    tokenStm = [None] * len(textSW)
    for i in range(0, len(textSW)):
        tokenStm[i]=pst.stem(textSW[i])
    return tokenStm
    #fdist = FreqDist(tokenStm)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def tokenizcionEspanol(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"'"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    #quitarHasTags
    
    ##textMin = filter(lambda x:x[0]!='#', word_tokenize(textPunt.lower()))
    textMin = word_tokenize(textPunt.lower())    
    
    textSW = [x for x in textMin if x not in stopWSP]
    textSW = [x for x in textSW if x not in word_rem['Palabras'].values] 
    stemmed_text = [stemmer.stem(i) for i in textSW]
    return stemmed_text
    #fdist = FreqDist(stemmed_text)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def seleccionTokenizacion(fila,var):
    if fila["lang"] == "en" :
        return tokenizcionIngles(str(fila[var]))
    elif fila["lang"] == "es" :
        return tokenizcionEspanol(str(fila[var]))


#RSS definicion de métodos
def parse(url):
    return feedparser.parse(url)

def get_source(parsed):
    feed= parsed['feed']
    return {
        'link': feed['link'],
        'title': feed['title'],
        'subtitle': feed['subtitle']
    }

def get_articles(parsed):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            articles.append({
                'id': entry['id'],
                'link': entry['link'],
                'title': entry['title'],
                'summary':entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def get_articles_resultado(parsed,fuenteRSS):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            #partDate =entry['published_parsed'].replace('(','')replace(')','').split(",")
            #almacenarFuente            
            # validacion si falta una dato            
            if 'summary' not in entry :
                entry['summary']=""
            if 'title' not in entry :
                entry['title']=""    
            articles.append({
                'fuenteRss':fuenteRSS,
                'idP': entry['id'],                
                'texto': entry['title']+". "+entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def getFechaFromRss(fechaStruct):
    #print (fechaStruct)
    if fechaStruct is None:
        #print ("entro")
        return datetime.now()
    else:
        return datetime.fromtimestamp(mktime(fechaStruct))

In [18]:
### Funciones para modelo de tópicos
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def impresionTopicos(mod,nPalabras,nTopicos):
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    for topico in topicos:
        print(topico)    
        
def impresionTopicosLimp(mod,nPalabras,nTopicos):
    #Impresion limpia de Tópicos
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    i=0
    for topico in topicos:
        i=i+1
        textoAux = re.sub(r'\W', ' ', str(topico))
        textoAux = re.sub(r'\s+[a-zA-Z]\s+', ' ', textoAux)
        textoAux = re.sub(r'[0-9]', ' ', textoAux)
        textoAux = re.sub(r'\s+', ' ', textoAux, flags=re.I)
        print(str(i),textoAux)

## Carga de los datos

In [ ]:
### Twitter
sql_select_Query = "select * from tesis.t_base_twitter"
base_twitter=pd.read_sql(sql_select_Query, connection)

In [ ]:
#Rss
sql_select_Query = "select * from tesis.t_base_rss"
base_rss=pd.read_sql(sql_select_Query, connection)

In [45]:
connection.close()

### TWITER

In [19]:
#sql_select_Query = "select * from tesis.t_base_twitter"
#base_twitter=pd.read_sql(sql_select_Query, connection)

In [20]:
base_twitter

,index,screen_name,id,id_str,name,location,Idioma,concepto,impacto,NoTweets
0,0,sergiodigra,NaN,NaN,None,None,es,None,NaN,100
1,1,Victor_Malagon,NaN,NaN,None,None,es,None,NaN,100
2,2,jscorrales,NaN,NaN,None,None,es,None,NaN,100
3,3,_minecogob,NaN,NaN,None,None,es,None,NaN,100
4,4,elEconomistaes,NaN,NaN,None,None,es,None,NaN,100
5,5,FT,NaN,NaN,None,None,en,None,NaN,100
6,6,rodriguezbraun,NaN,NaN,None,None,es,None,NaN,100
7,7,josecdiez,NaN,NaN,None,None,es,None,NaN,100
8,8,suanzes,NaN,NaN,None,None,es,None,NaN,100
9,9,wef,NaN,NaN,None,None,es,None,NaN,100


In [21]:
###TWEETER
# autenticación
# keys and tokens from the Twitter Dev Console 
consumer_key = '5BOcNnQsuIxDTBejlZ75xBqZ0'
consumer_secret = 'c6Cnx7SItN5Gda6K8hgtPo3tPyI18TU9eUh28jx3d4lDceewiG'
access_token = '927023579727884288-1jTaXF5wmUbX2NeYzUcSlmzXQMca4kU'
access_token_secret = 'Y62wUwgoWjfHGw5S0jlYH7w8ulmlRzaFcxbGjBP0wLin9'

#creación del objeto para extraer información
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#obtejo
api= tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
# si llega al cupo espera y continua


#df=pd.read_csv("AmigosTweetter_base_V2.csv", sep=';',encoding = "ISO-8859-1", engine='python', index_col="id")
    

In [22]:
#Cargar Tweets
tweets = []
fechaDelTweetAnterior=fechaHoy
    
#for index, row in df.iterrows():
for index, row in base_twitter.iterrows():
    #for amigo in df:
    i=0
    print(str(i)+" "+row['screen_name'])
    tweetsAmigo=tweepy.Cursor(api.user_timeline, screen_name=row['screen_name'], tweet_mode="extended").items(row['NoTweets'])
    for tweet in tweetsAmigo:
        tweets.append({
            'id':tweet.id,
            'full_text':tweet.full_text,
            'created_at':tweet.created_at,
            #'lang':row['Idioma'],
            'lang':tweet.lang,
            'id_fuente':row['screen_name']
        })
    i=i+1 

0 sergiodigra
0 Victor_Malagon
0 jscorrales
0 _minecogob
0 elEconomistaes
0 FT
0 rodriguezbraun
0 josecdiez
0 suanzes
0 wef
0 ecofeminita
0 epeconomia
0 RevistaDinero
0 bbcmundo
0 BluRadioCo
0 elespectador
0 RevistaSemana
0 ELTIEMPO
0 elcolombiano
0 lafm
0 NoticiasRCN
0 rcnradio
0 MailOnline
0 clarincom
0 NBCNews
0 rcnmundo
0 CaracolRadio
0 NoticiasCaracol
0 NTN24
0 NASA
0 PublimetroCol
0 Adcamach
0 Adriana_Guzman
0 agaviriau
0 AlbertoBernalLE
0 alhosu
0 amaldon19
0 Anamibanez
0 BancoRepublica
0 BvcColombia
0 CamiloDeGuzman
0 carlos_g_ospino
0 castellanosgd
0 CehernandezEcon
0 Cesaretamayo
0 estebanpie
0 Fedesarrollo
0 FocoEconomico
0 Fondosdepension
0 freddykastro
0 IECarballo
0 jagallegod
0 japebu
0 JavierMejiaC
0 jleibovi
0 Jllanosala
0 JoMalagon
0 JorgeARestrepo
0 JorgeATovar
0 JoseILOPEZ
0 jrobertoacosta1
0 juanf_vargas
0 juansrobledo
0 JusTributaria
0 LeopoldoTweets
0 LuisFelipeHenao
0 LuisFerMejia
0 mahofste
0 MantillaIgnacio
0 mariovalencia01
0 MauricioCard
0 Maurotov
0 Melende

In [23]:
tweetsDF=pd.DataFrame(tweets)
pd.set_option('display.max_colwidth', -1)
tweetsDFProcesado=tweetsDF.copy()
### limpiar texto
tweetsDFProcesado["full_text_Limpio"]=tweetsDFProcesado.full_text.apply(limpiar_texto)

In [24]:
tweetsDFProcesado.lang.value_counts()

es     7231
en     1287
und    205 
ca     21  
pt     18  
fr     10  
it     6   
ro     5   
de     4   
no     2   
in     2   
cs     2   
nl     1   
et     1   
eu     1   
fi     1   
tr     1   
tl     1   
hu     1   
Name: lang, dtype: int64

In [25]:
tweetsDFProcesado['SentSQDM']=tweetsDFProcesado.full_text_Limpio.apply(sentiment.sentiment)*2-1

In [26]:
#tweetsDFProcesado['SentTesis']=tweetsDFProcesado.full_text_Limpio.apply(seleccionAnalisis)
tweetsDFProcesado['SentTesis']=tweetsDFProcesado.apply(lambda row: seleccionAnalisis(row), axis=1)

In [42]:
tweetsDFProcesado.head(10)

,id,full_text,created_at,lang,id_fuente,full_text_Limpio,SentSQDM,SentTesis
0,1482183537482780674,En @AgendaCAF nos inspira la decisión de los Presidentes @IvanDuque @LassoGuillermo @NitoCortizo @CarlosAlvQ al avanzar en la consolidación en el Océano Pacifico del área protegida marina mas importante del planeta. Acompañamos decididamente esta iniciativa!,2022-01-15 02:51:07,es,sergiodigra,en nos inspira la decisión de los presidentes al avanzar en la consolidación en el océano pacifico del área protegida marina mas importante del planeta acompañamos decididamente esta iniciativa,-0.50,0.91
1,1482029273657577474,RT @chrisasinelli: Productivas reuniones del equipo de @agendacaf en DC para avanzar en la agenda verde. Gracias Andrew Griffin de @StateDe…,2022-01-14 16:38:08,es,sergiodigra,rt productivas reuniones del equipo de en dc para avanzar en la agenda verde gracias andrew griffin de,0.14,-0.73
2,1482024827565268995,"Agradezco al Presidente Electo 🇨🇱 @gabrielboric por la charla de hoy. Desde @AgendaCAF nuestro total compromiso por el éxito de su gobierno. Conversamos sobre acción climática, transición energética, diversificación matriz productiva e integración regional, entre otros temas.",2022-01-14 16:20:28,es,sergiodigra,agradezco al presidente electo por la charla de hoy desde nuestro total compromiso por el éxito de su gobierno conversamos sobre acción climática transición energética diversificación matriz productiva integración regional entre otros temas,-1.00,0.29
3,1480527866873921536,RT @antonioguterres: 2022 cannot be yet another year of disrupted education. \n\nNo effort should be spared to get children's education back…,2022-01-10 13:12:04,en,sergiodigra,rt cannot be yet another year of disrupted education no effort should be spared to get children education back,-0.56,-0.30
4,1480199590305570821,"RT @PaulaMorenoZ: Mi columna de hoy.Arranco 2022 pensando que hay que incomodarse,percibir la crueldad de la realidad cotidiana de las ""mi…",2022-01-09 15:27:37,es,sergiodigra,rt mi columna de hoy arranco pensando que hay que incomodarse percibir la crueldad de la realidad cotidiana de las mi,-1.00,-0.03
5,1480172132357267464,RT @Osgazu: Reconocer que existe el odioso racismo es el mejor comienzo para enfrentarlo! No inventar excusas ridículas cuando los hechos d…,2022-01-09 13:38:31,es,sergiodigra,rt reconocer que existe el odioso racismo es el mejor comienzo para enfrentarlo no inventar excusas ridículas cuando los hechos,-0.16,-0.90
6,1479840223861350406,"RT @AgendaCAF: Gracias a estas aprobaciones más 2,5 millones de personas cuenten con conexiones nuevas o mejoradas de agua potable y/o alca…",2022-01-08 15:39:38,es,sergiodigra,rt gracias estas aprobaciones más millones de personas cuenten con conexiones nuevas mejoradas de agua potable o alca,-0.99,0.21
7,1479626743178792962,"RT @trenesarginf: Para un tren inclusivo, con el financiamiento de @AgendaCAF, avanzamos con un Plan de Acción en Género para la renovación…",2022-01-08 01:31:20,es,sergiodigra,rt para un tren inclusivo con el financiamiento de avanzamos con un plan de acción en género para la renovación,0.52,-0.91
8,1479566734189289472,"RT @chrisasinelli: Iniciamos el año participando del primer taller estratégico sobre Género, Inclusión y Diversidad. Desde @agendacaf impul…",2022-01-07 21:32:53,es,sergiodigra,rt iniciamos el año participando del primer taller estratégico sobre género inclusión diversidad desde impul,-0.43,-0.23
9,1478811051777003521,RT @AgendaCAF: Convocatoria de CAF a investigaciones (hasta 15.000 USD) sobre beneficios y costos de acción ante el cambio climático y cuid…,2022-01-05 19:30:04,es,sergiodigra,rt convocatoria de caf investigaciones hasta usd sobre beneficios costos de acción ante el cambio climático cuid,-0.96,-0.37


# RSS

In [27]:
#sql_select_Query = "select * from tesis.t_base_rss"
#base_rss=pd.read_sql(sql_select_Query, connection)

In [29]:
base_rss[:8]

,index,id,FuentesRSS,lenguaje,Fuente,Pais,tema,Concepto,funciona,concepto_estandar,impacto,inter_local,DescxErrores
0,0,1.00,http://ep00.epimg.net/rss/tags/ultimas_noticias.xml,esp,el pais,venezuela,ultimo,Últimas noticias,None,actualidad,3,None,0
1,1,2.00,http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml,esp,el pais,venezuela,más leido,Noticias más vistas,None,actualidad,6,None,0
2,2,3.00,http://ep00.epimg.net/rss/economia/portada.xml,esp,el pais,colombia,economía,Portada de Economía,None,economía,10,None,0
3,3,4.00,https://www.clarin.com/rss/lo-ultimo/,esp,el clarín,argentina,lo-ultimo,"Home, lo último",None,actualidad,6,None,0
4,4,5.00,https://www.clarin.com/rss/economia/,esp,el clarín,argentina,economia,Economía,None,economía,10,None,0
5,5,6.00,https://www.elcomercio.com/feed/,esp,el comercio,perú,None,Lo último,None,actualidad,6,None,0
6,6,7.00,http://www.elcomercio.com/rss/actualidad,esp,el comercio,perú,actualidad,Actualidad,None,actualidad,6,None,1
7,7,8.00,http://cdn.eluniversal.com/rss/avances.xml,esp,el universal,venezuela,avances,"Avances, mala",None,actualidad,6,None,1


In [50]:
#RSS
##df=pd.read_csv("FuentesRss.csv", sep=';',encoding = "ISO-8859-1", engine='python', index_col="id")
#eliminar descartados
base_rss=base_rss[base_rss.DescxErrores != 1]
base_rss

##tokenTweetsFinalRSS=pd.DataFrame(columns =['word','frec','lang','id','created_at','id_fuente'])  
##tokenTweetsFinalRSS

#procesamiento de RSS
#dicResultado=[]
dicResultado=pd.DataFrame()
#dataFrameResultado = pd.DataFrame() 
bandera=0

for index, row in base_rss.iterrows(): 
    fuenteRss= row["FuentesRSS"]
    idioma=row["lenguaje"]
    #topico=row["concepto estandar"]
    pais=row["Pais"]
    
    #depurar antes de procesar
    print ("procesando fuente numero: "+str(index))
    print ( "fuenteRss:"+str(fuenteRss)+" idioma:"+str(idioma)+ " topico:"+str(pais) )   
    
    try:
        auxParse=parse(fuenteRss)
        articles1=get_articles_resultado(auxParse,fuenteRss)
        #articles1    
        #convertir a dataframe
        articles2=pd.DataFrame(articles1)
        #implementacion fechas a articulos
        articles2
        #depurar fecha recibida
        #print (articles2["published"])
    
        articles2["date"]=articles2["published"].apply(getFechaFromRss)
        articles2["lang"]=idioma
        
        articles2["texto_Limpio"]=articles2["texto"].apply(limpiar_texto)
        
        
        if idioma == "eng" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosIngles) 
        elif idioma == "esp" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosEspanol)
        #analisis sentimientos 2
        articles2['polaridad_SQDM']=articles2["texto_Limpio"].apply(sentiment.sentiment)*2-1
        bandera=bandera+1
        if bandera==1:
            dicResultado=articles2
        else:
            dicResultado=dicResultado.append(articles2)
        
        #dicResultado.append(articles2)
    except Exception as e:
        print(e)
    
#dataFrameResultado=pd.concat(dicResultado)

"""
#tokenizacion RSS
tokenRSSFinal=pd.DataFrame(columns =['word','frec','lang','idP','date','fuenteRss'])  
tokenRSSFinal
### tokenizacion
for index, row in dataFrameResultado.iterrows():
    #print(row)
    tokenRSSDF=seleccionTokenizacion(row,"texto")
    tokenRSSDF['lang']=row['lang']
    tokenRSSDF['idP']=row['idP']
    tokenRSSDF['date']=row['date']
    tokenRSSDF['fuenteRss']=row['fuenteRss']
    tokenRSSFinal=tokenRSSFinal.append(tokenRSSDF, ignore_index = True)
"""

#ajustar archivo
##dataFrameResultado=pd.concat(dicResultado)
##dataFrameResultado.pop("texto")
##dataFrameResultado.pop("published")

procesando fuente numero: 0
fuenteRss:http://ep00.epimg.net/rss/tags/ultimas_noticias.xml idioma:esp topico:venezuela
'published'
procesando fuente numero: 1
fuenteRss:http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml idioma:esp topico:venezuela
procesando fuente numero: 2
fuenteRss:http://ep00.epimg.net/rss/economia/portada.xml idioma:esp topico:colombia
procesando fuente numero: 3
fuenteRss:https://www.clarin.com/rss/lo-ultimo/ idioma:esp topico:argentina
procesando fuente numero: 4
fuenteRss:https://www.clarin.com/rss/economia/ idioma:esp topico:argentina
procesando fuente numero: 5
fuenteRss:https://www.elcomercio.com/feed/ idioma:esp topico:perú
procesando fuente numero: 8
fuenteRss:http://elnacional.com.do/feed/ idioma:esp topico:venezuela
procesando fuente numero: 9
fuenteRss:http://www.eltiempo.com/rss/economia.xml idioma:esp topico:colombia
procesando fuente numero: 10
fuenteRss:http://www.eltiempo.com/rss/deportes.xml idioma:esp topico:colombia
procesando fuente numero: 

'published'
procesando fuente numero: 78
fuenteRss:http://rssfeeds.usatoday.com/UsatodaycomNation-TopStories idioma:eng topico:usa
procesando fuente numero: 79
fuenteRss:http://feeds.washingtonpost.com/rss/rss_blogpost idioma:eng topico:usa
procesando fuente numero: 80
fuenteRss:https://www.eltiempo.com/rss/mundo_latinoamerica.xml idioma:esp topico:colombia
procesando fuente numero: 81
fuenteRss:https://www.eltiempo.com/rss/mundo_eeuu-y-canada.xml idioma:esp topico:colombia
procesando fuente numero: 82
fuenteRss:http://www.eltiempo.com/rss/mundo_europa.xml idioma:esp topico:colombia
procesando fuente numero: 83
fuenteRss:http://www.eltiempo.com/rss/mundo_medio-oriente.xml idioma:esp topico:colombia
procesando fuente numero: 84
fuenteRss:http://www.eltiempo.com/rss/mundo_asia.xml idioma:esp topico:colombia
procesando fuente numero: 85
fuenteRss:https://www.eltiempo.com/rss/mundo_africa.xml idioma:esp topico:colombia
procesando fuente numero: 86
fuenteRss:http://www.eltiempo.com/rss/econ

'\n#tokenizacion RSS\ntokenRSSFinal=pd.DataFrame(columns =[\'word\',\'frec\',\'lang\',\'idP\',\'date\',\'fuenteRss\'])  \ntokenRSSFinal\n### tokenizacion\nfor index, row in dataFrameResultado.iterrows():\n    #print(row)\n    tokenRSSDF=seleccionTokenizacion(row,"texto")\n    tokenRSSDF[\'lang\']=row[\'lang\']\n    tokenRSSDF[\'idP\']=row[\'idP\']\n    tokenRSSDF[\'date\']=row[\'date\']\n    tokenRSSDF[\'fuenteRss\']=row[\'fuenteRss\']\n    tokenRSSFinal=tokenRSSFinal.append(tokenRSSDF, ignore_index = True)\n'

In [52]:
dicResultado.sample(10)

,fuenteRss,idP,texto,published,date,lang,texto_Limpio,polaridad,polaridad_SQDM
18,http://ep00.epimg.net/rss/economia/portada.xml,https://elpais.com/economia/2020/11/25/mis_finanzas/1606321686_653551.html#?ref=rss&format=simple&link=guid,Vivienda en propiedad o de alquiler: última revisión antes del cierre fiscal. Los asesores fiscales repasan los elementos a vigilar para aminorar la próxima declaración de la renta en este ámbito,"(2020, 12, 5, 8, 21, 40, 5, 340, 0)",2020-12-05 08:21:40,esp,vivienda en propiedad de alquiler última revisión antes del cierre fiscal los asesores fiscales repasan los elementos vigilar para aminorar la próxima declaración de la renta en este ámbito,-0.45,-0.25
1,http://www.portafolio.co/rss/economia/gobierno,https://www.portafolio.co/economia/gobierno/que-pasa-si-no-alcanzo-a-inscribir-su-cedula-560544,Qué pasa si no alcanzó a inscribir su cédula para las elecciones.,"(2022, 1, 14, 23, 52, 33, 4, 14, 0)",2022-01-14 23:52:33,esp,qué pasa si no alcanzó inscribir su cédula para las elecciones,-0.99,-1.00
2,http://www.nytimes.com/services/xml/rss/nyt/World.xml,https://www.nytimes.com/2022/01/15/world/europe/bangladesh-london-brick-lane-gentrification.html,"Towers Rise Over London’s Brick Lane, Clouding Its Future. Luxury developments and rising rents have ignited concerns about gentrification and displacement in a district that has long been a sanctuary for minority communities.","(2022, 1, 15, 18, 8, 46, 5, 15, 0)",2022-01-15 18:08:46,eng,towers rise over london brick lane clouding its future luxury developments and rising rents have ignited concerns about gentrification and displacement in district that has long been sanctuary for minority communities,0.00,-0.63
5,http://www.portafolio.co/rss/negocios,https://www.portafolio.co/negocios/empresas/avicanna-lanza-dos-nuevas-formulaciones-a-base-de-cannabis-medicinal-560571,Avicanna lanza dos nuevas formulaciones a base de cannabis medicinal.,"(2022, 1, 15, 15, 39, 34, 5, 15, 0)",2022-01-15 15:39:34,esp,avicanna lanza dos nuevas formulaciones base de cannabis medicinal,-0.83,-0.12
0,https://feeds.a.dj.com/rss/RSSLifestyle.xml,SB11652343984522924106804588179883505889080,The Secret Power of Reconnecting With Old Friends. There’s a special boost that only old friends can give us. Here’s why we need it now.,"(2021, 11, 16, 18, 8, 0, 1, 320, 0)",2021-11-16 18:08:00,eng,the secret power of reconnecting with old friends there a special boost that only old friends can give us here why we need it now,0.89,0.36
12,http://elnacional.com.do/feed/,https://elnacional.com.do/?p=1317088,"Plausible. Conforme a la explicación del ministro de Energía y Minas, Antonio Almonte, es plausible el fideicomiso que se creó para la termoeléctrica de Punta Catalina. Además de continuar como propiedad del Estado, el mecanismo no solo propicia la despolitazación operativa y administrativa de las unidades, sino aportarle beneficios por la venta de energía. Sin poder [&#8230;]","(2022, 1, 15, 20, 8, 52, 5, 15, 0)",2022-01-15 20:08:52,esp,plausible conforme la explicación del ministro de energía minas antonio almonte es plausible el fideicomiso que se creó para la termoeléctrica de punta catalina además de continuar como propiedad del estado el mecanismo no solo propicia la despolitazación operativa administrativa de las unidades sino aportarle beneficios por la venta de energía sin poder,-0.27,-1.00
3,http://feeds.washingtonpost.com/rss/rss_monkey-cage,https://www.washingtonpost.com/politics/2022/01/14/greene-social-media-misinformation/,"Twitter banned Marjorie Taylor Greene. That may not hurt her much.. She’s gaining followers and 'likes’ on other social media platforms, our research finds.","(2022, 1, 14, 12, 0, 57, 4, 14, 0)",2022-01-14 12:00:57,eng,twitter banned marjorie taylor greene that may not hurt her much she gaining followers and likes on other social media platforms our research finds,0.66,-0.66
5,https://www.clarin.com/rss/lo-ultimo/,https://www.clarin.com/deportes/minuto-m

### Guardar en la base de datos

In [48]:
import mysql.connector
import sqlalchemy

database_username = 'root'
database_password = 'Cidifo03%%'
database_ip       = '127.0.0.1:3307'
database_name     = 'tesis'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()

In [49]:
tweetsDFProcesado.drop(columns=['full_text']).to_sql(con=database_connection, name='t_base_tweets_new', if_exists='replace',chunksize=100)

In [53]:
dicResultado.drop(columns=['published','texto']).to_sql(con=database_connection, name='t_base_rss_new', if_exists='replace',chunksize=100)

In [54]:
database_connection.close()

### Próximos pasos

* Definir si se va aguardar información de tokenización
* Definir si se van a definir tópicos generales o por fuente para segmentar los textos y buscar correlaciones.
* try cath RSS

## Recortes

## Carga de los datos

In [ ]:
#TRM = 'TRM2020.csv'
trm_df=pd.read_csv('TRM2020.csv', sep=';',encoding = "ISO-8859-1", engine='python', decimal=",")

In [ ]:
trm_df.head()

In [ ]:
df_Rss = pd.read_excel (r'backupT.xlsx', sheet_name='Hoja3')

In [ ]:
df_Rss.head()

In [ ]:
df_Base_Rss = pd.read_excel (r'backupT.xlsx', sheet_name='Hoja1')

In [ ]:
df_Base_Rss.head()

In [ ]:
df_Tweets = pd.read_excel (r'backupT.xlsx', sheet_name='Hoja7')

In [ ]:
df_Tweets.head()

In [ ]:
df_Base_Tweets = pd.read_excel (r'backupT.xlsx', sheet_name='Hoja2')

In [ ]:
df_Base_Tweets.head()

In [ ]:
### Carga de TRM
#df_trm=pd.read_csv(ubicacion, sep=';',encoding = "ISO-8859-1", engine='python', decimal=",")
trm_df=pd.read_csv('TRM2020.csv', sep=';',encoding = "ISO-8859-1", engine='python', decimal=",")

In [ ]:
trm_df['date']=pd.to_datetime(trm_df['date'])

In [ ]:
#trm_df.set_index('date',inplace=True)

In [ ]:
trm_df.head(10)

In [ ]:
trm_df['date_ini']=trm_df['date']
trm_df['date_fin']=trm_df['date']

In [ ]:
flag=0
for index, row in trm_df.iterrows():
    if row['Dif Cero'] == 1 :
        if flag==0 :
            index_ini=index
            flag=1
    else :        
        if flag==1:
            dia_fin=row['date']
            index_fin=index
            flag=0
            #trm_df[trm_df['date'].between(dia_ini,dia_fin)]['date_fin']=dia_fin
            for i in range(index_ini,index_fin+1):
                #print(i,dia_fin)
                trm_df.loc[i,'date_fin']=dia_fin

In [ ]:
calendario=trm_df[['date','date_fin']]
calendario.head(10)

In [ ]:
calendario.dtypes

### Resumir fuentes con medianas

#### Quitar Cero

In [ ]:
df_Rss=df_Rss[df_Rss['polaridad']!=0]

In [ ]:
df_Rss['dateF']=pd.to_datetime(df_Rss['date']).dt.date

In [ ]:
df_Rss['dateF']=pd.to_datetime(df_Rss['dateF'])

In [ ]:
df_Rss.head(10)

In [ ]:
df_Rss.dtypes

In [ ]:
df_Rss=df_Rss.merge(calendario, left_on='dateF', right_on='date', how='left')

In [ ]:
df_Tweets.head(10)

In [ ]:
df_Tweets=df_Tweets[df_Tweets['polaridad']!=0]

In [ ]:
df_Tweets['dateF']=pd.to_datetime(df_Tweets['created_at']).dt.date
df_Tweets['dateF']=pd.to_datetime(df_Tweets['dateF'])

In [ ]:
df_Tweets=df_Tweets.merge(calendario, left_on='dateF', right_on='date', how='left')

##### Describir Bases

In [ ]:
df_Rss['polaridad'].describe()

In [ ]:
df_Tweets['polaridad'].describe()

## Manejo de fechas sin operaciones

In [ ]:
#dias_sin_operacion.head()

¿correlacionan mejor sin contar los días sin operación?

In [ ]:
trm_df.head(20)

#### Agrupación fuente día

In [ ]:
res_Rss=df_Rss.groupby(['fuenteRss','date_fin'])['polaridad'].median()
res_Rss=res_Rss.to_frame()
res_Rss.reset_index(inplace=True)

In [ ]:
res_Rss.head(10)

In [ ]:
df_Tweets.head(2)

In [ ]:
res_Tweets=df_Tweets.groupby(['id_fuente','date_fin'])['polaridad'].median()
res_Tweets=res_Tweets.to_frame()
res_Tweets.reset_index(inplace=True)

In [ ]:
res_Tweets.head(10)

###  revisar rangos de fechas dataSets

In [ ]:
print('res_Rss',res_Rss['date_fin'].min(),res_Rss['date_fin'].max())
print('res_Tweets',res_Tweets['date_fin'].min(),res_Tweets['date_fin'].max())
print('trm_df',trm_df['date_fin'].min(),trm_df['date_fin'].max())

In [ ]:
res_Rss=res_Rss[(res_Rss['date_fin']>='2020-10-15')&(res_Rss['date_fin']<='2021-06-15')]

In [ ]:
res_Rss=res_Rss.merge(trm_df[['date','Diferencia']],left_on='date_fin', right_on='date', how='left')

In [ ]:
res_Rss.head()

In [ ]:
res_Tweets=res_Tweets[(res_Tweets['date_fin']>='2020-10-15')&(res_Tweets['date_fin']<='2021-06-15')]

In [ ]:
res_Tweets=res_Tweets.merge(trm_df[['date','Diferencia']],left_on='date_fin', right_on='date', how='left')

In [ ]:
res_Tweets.head()

###  Tablas correlacion

In [ ]:
res_Rss_resultados=res_Rss[['fuenteRss']].value_counts().to_frame()

In [ ]:
corr_Rss=res_Rss.groupby('fuenteRss')[['polaridad','Diferencia']].corr()

In [ ]:
corr_Rss.reset_index(inplace=True)
corr_Rss=corr_Rss[corr_Rss['level_1']=='Diferencia']
corr_Rss.drop(columns=['Diferencia','level_1'], inplace=True)

In [ ]:
res_Rss_resultados=res_Rss_resultados.merge(corr_Rss, left_on='fuenteRss', right_on='fuenteRss', how='left')

In [ ]:
res_Rss_resultados.columns=['fuenteRss','cuenta','corr']

In [ ]:
res_Rss_resultados.sort_values('corr')

In [ ]:
res_Tweets_resultados=res_Tweets[['id_fuente']].value_counts().to_frame()

In [ ]:
corr_Tweets=res_Tweets.groupby('id_fuente')[['polaridad','Diferencia']].corr()

In [ ]:
corr_Tweets.reset_index(inplace=True)
corr_Tweets=corr_Tweets[corr_Tweets['level_1']=='Diferencia']
corr_Tweets.drop(columns=['Diferencia','level_1'], inplace=True)

In [ ]:
res_Tweets_resultados=res_Tweets_resultados.merge(corr_Tweets, left_on='id_fuente', right_on='id_fuente', how='left')
res_Tweets_resultados.columns=['id_fuente','cuenta','corr']

In [ ]:
res_Tweets_resultados.sort_values('corr')

###  Validar Correlaciones

# Pasos a Seguir
3. Validar correlación con Lags
4. Validar correlacion spearman